In [1]:
!pip install qiskit qiskit_aer

In [2]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_bloch_multivector
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def qft_rotations(qc, n):
    """Apply QFT rotations recursively on the first n qubits."""
    if n == 0:
        return qc
    n -= 1
    qc.h(n)
    for qubit in range(n):
        qc.cp(np.pi / 2**(n - qubit), qubit, n)
    qft_rotations(qc, n)
    return qc

In [4]:
def swap_registers(qc, n):
    """Swap qubits to reverse their order."""
    for qubit in range(n // 2):
        qc.swap(qubit, n - qubit - 1)
    return qc

In [5]:
def qft_circuit(n):
    """Builds an n-qubit Quantum Fourier Transform circuit."""
    qc = QuantumCircuit(n)
    qft_rotations(qc, n)
    swap_registers(qc, n)
    qc.name = "QFT"
    return qc

In [6]:
def run_qft(n):
    """Executes QFT circuit and displays Bloch sphere representation."""
    qc = qft_circuit(n)

    # Create a simulator and save the statevector explicitly
    simulator = AerSimulator(method='statevector')
    qc.save_statevector()

    # Transpile and run
    compiled = transpile(qc, simulator)
    result = simulator.run(compiled).result()

    # Retrieve statevector safely
    statevector = result.data(0)["statevector"]

    # Display results
    plot_bloch_multivector(statevector)
    plt.show()
    print(qc.draw(output='text'))

if __name__ == "__main__":
    n_qubits = 3
    print(f"Running Quantum Fourier Transform on {n_qubits} qubits...")
    run_qft(n_qubits)

Running Quantum Fourier Transform on 3 qubits...
                                          ┌───┐    statevector 
q_0: ──────■──────────────────────■───────┤ H ├─X───────░──────
           │                ┌───┐ │P(π/2) └───┘ │       ░      
q_1: ──────┼────────■───────┤ H ├─■─────────────┼───────░──────
     ┌───┐ │P(π/4)  │P(π/2) └───┘               │       ░      
q_2: ┤ H ├─■────────■───────────────────────────X───────░──────
     └───┘                                              ░      


**TASK 1**

In [7]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_bloch_multivector
import numpy as np
import matplotlib.pyplot as plt


def qft_rotations(qc, n):
    """Apply QFT rotations recursively on the first n qubits."""
    if n == 0:
        return qc
    n -= 1
    qc.h(n)
    for qubit in range(n):
        qc.cp(np.pi / 2 ** (n - qubit), qubit, n)
    qft_rotations(qc, n)
    return qc


def swap_registers(qc, n):
    """Swap qubits to reverse their order."""
    for qubit in range(n // 2):
        qc.swap(qubit, n - qubit - 1)
    return qc


def qft_circuit(n):
    """Builds an n-qubit Quantum Fourier Transform circuit."""
    qc = QuantumCircuit(n)
    qft_rotations(qc, n)
    swap_registers(qc, n)
    qc.name = "QFT"
    return qc


def run_qft(n):
    """Executes QFT circuit and displays Bloch sphere representation."""
    qc = qft_circuit(n)

    simulator = AerSimulator(method='statevector')
    qc.save_statevector()

    compiled = transpile(qc, simulator)
    result = simulator.run(compiled).result()
    statevector = result.data(0)["statevector"]

    print(f"\n--- QFT on {n} Qubits ---")
    print(qc.draw(output='text'))

    plot_bloch_multivector(statevector)
    plt.show()


if __name__ == "__main__":
    for n_qubits in [2, 3, 4]:
        run_qft(n_qubits)



--- QFT on 2 Qubits ---
                   ┌───┐    statevector 
q_0: ──────■───────┤ H ├─X───────░──────
     ┌───┐ │P(π/2) └───┘ │       ░      
q_1: ┤ H ├─■─────────────X───────░──────
     └───┘                       ░      

--- QFT on 3 Qubits ---
                                          ┌───┐    statevector 
q_0: ──────■──────────────────────■───────┤ H ├─X───────░──────
           │                ┌───┐ │P(π/2) └───┘ │       ░      
q_1: ──────┼────────■───────┤ H ├─■─────────────┼───────░──────
     ┌───┐ │P(π/4)  │P(π/2) └───┘               │       ░      
q_2: ┤ H ├─■────────■───────────────────────────X───────░──────
     └───┘                                              ░      

--- QFT on 4 Qubits ---
                                                                          ┌───┐»
q_0: ──────■───────────────────────────────■──────────────────────■───────┤ H ├»
           │                               │                ┌───┐ │P(π/2) └───┘»
q_1: ──────┼────────■────────

**TASK 2**

In [8]:
def inverse_qft_rotations(qc, n):
    """Apply inverse QFT rotations recursively on the first n qubits."""
    if n == 0:
        return qc
    n -= 1

    # Recursive call
    inverse_qft_rotations(qc, n)

    # Apply inverse controlled-phase gates
    for qubit in range(n):
        qc.cp(-np.pi / 2 ** (n - qubit), qubit, n)
    qc.h(n)
    return qc


def swap_registers(qc, n):
    """Swap qubits to reverse their order."""
    for qubit in range(n // 2):
        qc.swap(qubit, n - qubit - 1)
    return qc


def inverse_qft_circuit(n):
    """Builds an n-qubit Inverse Quantum Fourier Transform circuit."""
    qc = QuantumCircuit(n)
    swap_registers(qc, n)
    inverse_qft_rotations(qc, n)
    qc.name = "Inverse QFT"
    return qc


def run_inverse_qft(n):
    """Executes inverse QFT circuit and displays Bloch sphere representation."""
    qc = inverse_qft_circuit(n)

    simulator = AerSimulator(method='statevector')
    qc.save_statevector()

    compiled = transpile(qc, simulator)
    result = simulator.run(compiled).result()
    statevector = result.data(0)["statevector"]

    print(f"\n--- Inverse QFT on {n} Qubits ---")
    print(qc.draw(output='text'))

    plot_bloch_multivector(statevector)
    plt.show()


if __name__ == "__main__":
    for n_qubits in [2, 3]:
        run_inverse_qft(n_qubits)


--- Inverse QFT on 2 Qubits ---
        ┌───┐                statevector 
q_0: ─X─┤ H ├─■───────────────────░──────
      │ └───┘ │P(-π/2) ┌───┐      ░      
q_1: ─X───────■────────┤ H ├──────░──────
                       └───┘      ░      

--- Inverse QFT on 3 Qubits ---
        ┌───┐                                         statevector 
q_0: ─X─┤ H ├─■──────────────■─────────────────────────────░──────
      │ └───┘ │P(-π/2) ┌───┐ │                             ░      
q_1: ─┼───────■────────┤ H ├─┼─────────■───────────────────░──────
      │                └───┘ │P(-π/4)  │P(-π/2) ┌───┐      ░      
q_2: ─X──────────────────────■─────────■────────┤ H ├──────░──────
                                                └───┘      ░      


**TASK 3**

In [9]:
from qiskit.visualization import plot_histogram
def qft_rotations(qc, n):
    if n == 0:
        return qc
    n -= 1
    qc.h(n)
    for qubit in range(n):
        qc.cp(np.pi / 2 ** (n - qubit), qubit, n)
    qft_rotations(qc, n)
    return qc


def swap_registers(qc, n):
    for qubit in range(n // 2):
        qc.swap(qubit, n - qubit - 1)
    return qc


def inverse_qft(qc, n):
    """Inverse QFT for n qubits."""
    swap_registers(qc, n)
    for qubit in range(n // 2):
        qc.swap(qubit, n - qubit - 1)
    for j in range(n // 2):
        qc.swap(j, n - j - 1)
    for qubit in range(n):
        for k in range(qubit):
            qc.cp(-np.pi / 2 ** (qubit - k), k, qubit)
        qc.h(qubit)
    return qc


def controlled_unitary(qc, control_qubit, target_qubit, theta):
    """Applies controlled-U gate: U = phase rotation by theta."""
    qc.cp(2 * np.pi * theta, control_qubit, target_qubit)
    return qc


def phase_estimation(theta, counting_qubits=3):
    """Quantum Phase Estimation demo circuit."""
    qc = QuantumCircuit(counting_qubits + 1, counting_qubits)

    # Step 1: Initialize eigenstate |ψ⟩ on target
    qc.x(counting_qubits)  # |1>

    # Step 2: Hadamards on counting qubits
    qc.h(range(counting_qubits))

    # Step 3: Controlled-U^2^j gates
    for j in range(counting_qubits):
        controlled_unitary(qc, j, counting_qubits, theta * (2 ** j))

    # Step 4: Apply inverse QFT to counting qubits
    inverse_qft(qc, counting_qubits)

    # Step 5: Measure counting qubits
    qc.measure(range(counting_qubits), range(counting_qubits))

    return qc


def run_phase_estimation(theta):
    simulator = AerSimulator()
    qc = phase_estimation(theta)
    compiled = transpile(qc, simulator)
    result = simulator.run(compiled, shots=1024).result()
    counts = result.get_counts()

    print("\n--- Quantum Phase Estimation ---")
    print(qc.draw(output='text'))

    plot_histogram(counts)
    plt.show()

    # Convert most likely bitstring to phase
    most_likely = max(counts, key=counts.get)
    phase = int(most_likely, 2) / (2 ** 3)
    print(f"Estimated phase = {phase:.3f}")
    print(f"Actual phase = {theta:.3f}\n")


if __name__ == "__main__":
    for theta in [0.125, 0.25, 0.375]:
        run_phase_estimation(theta)


--- Quantum Phase Estimation ---
     ┌───┐                                  ┌───┐                         »
q_0: ┤ H ├─■────────────────────────X──X──X─┤ H ├─■──────────────■────────»
     ├───┤ │                        │  │  │ └───┘ │P(-π/2) ┌───┐ │        »
q_1: ┤ H ├─┼────────■───────────────┼──┼──┼───────■────────┤ H ├─┼────────»
     ├───┤ │        │               │  │  │                └───┘ │P(-π/4) »
q_2: ┤ H ├─┼────────┼────────■──────X──X──X──────────────────────■────────»
     ├───┤ │P(π/4)  │P(π/2)  │P(π)                                        »
q_3: ┤ X ├─■────────■────────■────────────────────────────────────────────»
     └───┘                                                                »
c: 3/═════════════════════════════════════════════════════════════════════»
                                                                          »
«               ┌─┐           
«q_0: ──────────┤M├───────────
«               └╥┘     ┌─┐   
«q_1: ─■─────────╫──────┤M├───
«     

**TASK 4**

In [10]:
!pip install pylatexenc


In [11]:
#  TASK 4: Measure Output States after QFT (Text-only, no plots)
# ---------------------------------------------------------------
# This prints clear QFT circuits and probability counts for each input.

from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
import numpy as np


# ------------------------
# QFT construction
# ------------------------
def qft_rotations(qc, n):
    if n == 0:
        return qc
    n -= 1
    qc.h(n)
    for qubit in range(n):
        qc.cp(np.pi / 2 ** (n - qubit), qubit, n)
    qft_rotations(qc, n)
    return qc


def swap_registers(qc, n):
    for qubit in range(n // 2):
        qc.swap(qubit, n - qubit - 1)
    return qc


def qft_circuit(n):
    qc = QuantumCircuit(n)
    qft_rotations(qc, n)
    swap_registers(qc, n)
    return qc


# ------------------------
# QFT + Measurement Runner
# ------------------------
def run_qft_with_measurement(n, input_state=None):
    """Run QFT with optional initial state and print measurement results."""
    qc = QuantumCircuit(n, n)

    # --- Input Preparation ---
    if input_state == "plus":
        qc.h(range(n))
    elif isinstance(input_state, int):
        for i in range(n):
            if (input_state >> i) & 1:
                qc.x(i)

    # --- Apply QFT and Measure ---
    qc.compose(qft_circuit(n), inplace=True)
    qc.measure(range(n), range(n))

    # --- Simulate with Aer ---
    simulator = Aer.get_backend("aer_simulator")
    compiled = transpile(qc, simulator)
    result = simulator.run(compiled, shots=2048, seed_simulator=42).result()
    counts = result.get_counts()

    # --- Print Results ---
    print(f"\n QFT Measurement Results (n={n}, input={input_state})")
    print("=" * 60)
    print(qc.draw(output="text"))
    print("\nCounts:")
    for bitstring, count in sorted(counts.items()):
        print(f"  {bitstring}: {count}")
    print("=" * 60)


# ------------------------
# Run Examples
# ------------------------
print(" Running QFT on |000⟩ ...")
run_qft_with_measurement(3, input_state=None)

print("\n Running QFT on |+++⟩ ...")
run_qft_with_measurement(3, input_state="plus")

print("\n Running QFT on |101⟩ ...")
run_qft_with_measurement(3, input_state=5)


 Running QFT on |000⟩ ...

 QFT Measurement Results (n=3, input=None)
                                          ┌───┐   ┌─┐   
q_0: ──────■──────────────────────■───────┤ H ├─X─┤M├───
           │                ┌───┐ │P(π/2) └┬─┬┘ │ └╥┘   
q_1: ──────┼────────■───────┤ H ├─■────────┤M├──┼──╫────
     ┌───┐ │P(π/4)  │P(π/2) └───┘          └╥┘  │  ║ ┌─┐
q_2: ┤ H ├─■────────■───────────────────────╫───X──╫─┤M├
     └───┘                                  ║      ║ └╥┘
c: 3/═══════════════════════════════════════╩══════╩══╩═
                                            1      0  2 

Counts:
  000: 249
  001: 261
  010: 242
  011: 252
  100: 259
  101: 262
  110: 286
  111: 237

 Running QFT on |+++⟩ ...

 QFT Measurement Results (n=3, input=plus)
     ┌───┐                                     ┌───┐   ┌─┐   
q_0: ┤ H ├──────■──────────────────────■───────┤ H ├─X─┤M├───
     ├───┤      │                ┌───┐ │P(π/2) └┬─┬┘ │ └╥┘   
q_1: ┤ H ├──────┼────────■───────┤ H ├─■────────┤M├──┼──╫────
 

**TASK 5**

In [12]:
# TASK 5: Inverse QFT Verification
# -----------------------------------------------

from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
import numpy as np


# ------------------------
# QFT and Inverse QFT
# ------------------------
def qft_rotations(qc, n):
    """Apply QFT rotations recursively."""
    if n == 0:
        return qc
    n -= 1
    qc.h(n)
    for qubit in range(n):
        qc.cp(np.pi / 2 ** (n - qubit), qubit, n)
    qft_rotations(qc, n)
    return qc


def swap_registers(qc, n):
    """Swap qubits to reverse their order."""
    for qubit in range(n // 2):
        qc.swap(qubit, n - qubit - 1)
    return qc


def qft_circuit(n):
    qc = QuantumCircuit(n)
    qft_rotations(qc, n)
    swap_registers(qc, n)
    return qc


def inverse_qft(qc, n):
    """Apply the inverse QFT (IQFT) to the circuit."""
    swap_registers(qc, n)
    for qubit in range(n):
        for k in range(qubit):
            qc.cp(-np.pi / 2 ** (qubit - k), k, qubit)
        qc.h(qubit)
    return qc


# ------------------------
# Combined QFT → IQFT Test
# ------------------------
def run_qft_inverse_test(n, input_state=None):
    """Apply QFT then IQFT, measure, and verify recovery."""
    qc = QuantumCircuit(n, n)

    # Prepare input state
    if input_state == "plus":
        qc.h(range(n))
    elif isinstance(input_state, int):
        for i in range(n):
            if (input_state >> i) & 1:
                qc.x(i)

    # Apply QFT
    qc.compose(qft_circuit(n), inplace=True)

    # Apply Inverse QFT
    inverse_qft(qc, n)

    # Measure final state
    qc.measure(range(n), range(n))

    # Simulate
    simulator = Aer.get_backend("aer_simulator")
    compiled = transpile(qc, simulator)
    result = simulator.run(compiled, shots=1024, seed_simulator=42).result()
    counts = result.get_counts()

    # Print results
    print(f"\n QFT → Inverse QFT Test (n={n}, input={input_state})")
    print("=" * 60)
    print(qc.draw(output="text"))
    print("\nCounts:")
    for bitstring, count in sorted(counts.items()):
        print(f"  {bitstring}: {count}")
    print("=" * 60)

    # Verify recovery
    max_outcome = max(counts, key=counts.get)
    print(f" Most frequent measurement: |{max_outcome}⟩ (original recovered)\n")


# ------------------------
# Run Examples
# ------------------------
print(" Testing QFT → Inverse QFT on |000⟩")
run_qft_inverse_test(3, input_state=None)

print("\n Testing QFT → Inverse QFT on |101⟩")
run_qft_inverse_test(3, input_state=5)

print("\n Testing QFT → Inverse QFT on |+++⟩ (superposition)")
run_qft_inverse_test(3, input_state="plus")


 Testing QFT → Inverse QFT on |000⟩

 QFT → Inverse QFT Test (n=3, input=None)
                                          ┌───┐      ┌───┐               »
q_0: ──────■──────────────────────■───────┤ H ├─X──X─┤ H ├─■─────────────»
           │                ┌───┐ │P(π/2) └───┘ │  │ └───┘ │P(-π/2) ┌───┐»
q_1: ──────┼────────■───────┤ H ├─■─────────────┼──┼───────■────────┤ H ├»
     ┌───┐ │P(π/4)  │P(π/2) └───┘               │  │                └───┘»
q_2: ┤ H ├─■────────■───────────────────────────X──X─────────────────────»
     └───┘                                                               »
c: 3/════════════════════════════════════════════════════════════════════»
                                                                         »
«                         ┌─┐           
«q_0: ─■──────────────────┤M├───────────
«      │                  └╥┘     ┌─┐   
«q_1: ─┼─────────■─────────╫──────┤M├───
«      │P(-π/4)  │P(-π/2)  ║ ┌───┐└╥┘┌─┐
«q_2: ─■─────────■─────────╫─┤ H ├─╫─┤M├
